In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
FOLDERNAME = "人工智能/Lab5"
assert FOLDERNAME is not None, "[!] Enter the foldername."
%cd drive/My\ Drive
%cp -r $FOLDERNAME ../../
%cd ../../Lab5

Mounted at /content/drive
/content/drive/My Drive
/content/Lab5


In [85]:
import shutil
import pandas as pd
import os
import numpy as np
import torch                    
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm.auto import tqdm

In [3]:
def mkdir_(path):
  if not os.path.exists(path):
    os.mkdir(path)

In [4]:
data_path = "data/"
train_path = "train.txt"
test_path = "test_without_label.txt"

In [5]:
def tag2label(tag):
    if tag == "negative": return 0
    elif tag == "neutral": return 1
    else: return 2

def get_label_dict(path):
  with open(path, 'r') as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines[1:]]
  dict_ = {}
  for line in lines:
    raw_data = line.split(",")
    dict_[raw_data[0]] = tag2label(raw_data[1])
  return dict_

def get_test_text(path):
  with open(path, 'r') as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines[1:]]
  return [line.split(",")[0] for line in lines]

def get_text(path):
    with open(path, 'r', errors='ignore') as f:
        line = f.readline()
    return line.strip()

In [6]:
label_dict = get_label_dict(train_path)
label_dict_keys = list(label_dict.keys())

In [8]:
img_data = [['id', 'label', 'img']]
for id_ in label_dict_keys:
  img_data.append([int(id_), int(label_dict[id_]), data_path+id_+".jpg"])
img_data = pd.DataFrame(img_data[1:], columns=img_data[0])

In [9]:
print(img_data)

        id  label            img
0     4597      0  data/4597.jpg
1       26      1    data/26.jpg
2     4383      0  data/4383.jpg
3      212      2   data/212.jpg
4     2626      2  data/2626.jpg
...    ...    ...            ...
3995  3944      0  data/3944.jpg
3996   945      2   data/945.jpg
3997   695      2   data/695.jpg
3998  4874      0  data/4874.jpg
3999   246      0   data/246.jpg

[4000 rows x 3 columns]


In [10]:
pic_path = "pic"
label_path = ["0","1","2"]
pic_train_path = "train"
pic_validate_path = "validate"

In [22]:
mkdir_(pic_path)
mkdir_(pic_path+"/"+pic_train_path)
mkdir_(pic_path+"/"+pic_validate_path)
for i in label_path:
  mkdir_(pic_path+"/train/"+i)
  mkdir_(pic_path+"/validate/"+i)

In [12]:
img_data.iloc[0].id

4597

In [13]:
kf = KFold(n_splits=10,shuffle=True)
for train_index, validate_index in kf.split(img_data):
  img_train = pd.DataFrame(img_data, index=train_index)
  img_validate = pd.DataFrame(img_data, index=validate_index)
  break

In [14]:
print(img_train)

        id  label            img
0     4597      0  data/4597.jpg
1       26      1    data/26.jpg
2     4383      0  data/4383.jpg
3      212      2   data/212.jpg
4     2626      2  data/2626.jpg
...    ...    ...            ...
3995  3944      0  data/3944.jpg
3996   945      2   data/945.jpg
3997   695      2   data/695.jpg
3998  4874      0  data/4874.jpg
3999   246      0   data/246.jpg

[3600 rows x 3 columns]


In [23]:
def copy_img(dataFrame, suffix, option):
  for i in range(len(dataFrame)):
    target_path = pic_path+"/"+suffix+"/"+str(dataFrame.iloc[i].label)
    target_path = pic_path+"/"+suffix+"/"
    source_path = dataFrame.iloc[i].img
    shutil.copy(source_path, target_path)

In [16]:
copy_img(img_train, pic_train_path)
copy_img(img_validate, pic_validate_path)

In [46]:
train_transform = transforms.Compose([
  transforms.Resize((224,224)),
  transforms.RandomHorizontalFlip(p=0.5),
  transforms.RandomVerticalFlip(p=0.5),
  transforms.ToTensor(),
  transforms.Normalize(
      mean=[0.485,0.456,0.406],
      std=[0.229,0.224,0.225]
  )
])
validate_transform = transforms.Compose([
  transforms.Resize((224,224)),
  transforms.ToTensor(),
  transforms.Normalize(
      mean=[0.485,0.456,0.406],
      std=[0.229,0.224,0.225]
  )
])

In [47]:
img_train_datasets = datasets.ImageFolder(pic_path+"/"+pic_train_path, transform=train_transform)
img_validate_datasets = datasets.ImageFolder(pic_path+"/"+pic_validate_path, transform=validate_transform)

In [48]:
img_train_loader = DataLoader(img_train_datasets, batch_size=16, shuffle=True, num_workers=0)
img_validate_loader = DataLoader(img_validate_datasets, batch_size=16, shuffle=True, num_workers=0)

In [50]:
def ret_model(pretrained=True, fine_tune=True, num_classes=3):
  model = models.resnet34(pretrained=pretrained)
  for params in model.parameters():
    params.requires_grad = fine_tune
  model.fc = nn.Linear(512, num_classes)
  return model

In [51]:
lr = 1e-4
epochs = 10
if torch.cuda.is_available():
  print("Use Cuda")
  device = 'cuda'
else:
  print("Use CPU")
  device = 'cpu'
model = ret_model(pretrained=True, fine_tune=False, num_classes=len(img_train_datasets.classes)).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

Use Cuda


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [62]:
def img_train(model, train_loader, optimizer, lossfn):
  model.train()
  print('Training:')
  train_running_loss = 0.0
  train_running_cor = 0
  cnt = 0
  for i, data in tqdm(enumerate(train_loader), total=len(train_loader)):
    cnt += 1
    img, labels = data
    img = img.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = model(img)
    loss = lossfn(outputs, labels)
    train_running_loss += loss.item()
    _, preds = torch.max(outputs.data, dim=1)
    train_running_cor += (preds == labels).sum().item()
    loss.backward()
    optimizer.step()
  epoch_loss = round(train_running_loss / cnt, 4)
  epoch_acc = round(train_running_cor / len(train_loader.dataset), 4)
  print("Train Loss: " + str(epoch_loss))
  print("Train Accuracy: " + str(epoch_acc))
  return epoch_loss, epoch_acc
  
def img_validate(model, validate_loader, optimizer, lossfn, num_classes):
  model.eval()
  print('Validating:')
  valid_running_loss = 0.0
  valid_running_cor = 0
  cnt = 0
  with torch.no_grad():
    for _, data in tqdm(enumerate(validate_loader), total=len(validate_loader)):
      cnt += 1
      img, labels = data
      img = img.to(device)
      labels = labels.to(device)
      optimizer.zero_grad()
      outputs = model(img)
      loss = lossfn(outputs, labels)
      valid_running_loss += loss.item()
      _, preds = torch.max(outputs.data, dim=1)
      valid_running_cor += (preds == labels).sum().item()

  epoch_loss = round(valid_running_loss / cnt, 4)
  epoch_acc = round(valid_running_cor / len(validate_loader.dataset), 4)
  print("Validate Loss: " + str(epoch_loss))
  print("Validate Accuracy: " + str(epoch_acc))
  return epoch_loss, epoch_acc


In [63]:
img_train_loss, img_train_acc = [], []
img_validate_loss, img_validate_acc = [], []
for epoch in range(epochs):
  print("Epoch " + str(epoch+1) + " of " + str(epochs))
  train_epoch_loss, train_epoch_acc = img_train(model, img_train_loader, optimizer, criterion)
  validate_epoch_loss, validate_epoch_acc = img_validate(model, img_validate_loader, optimizer, criterion, 3)
  img_train_loss.append(train_epoch_loss)
  img_train_acc.append(train_epoch_acc)
  img_validate_loss.append(validate_epoch_loss)
  img_validate_acc.append(validate_epoch_acc)
  print('-'*50)
torch.save(model, "save_model2.pth")  

Epoch 1 of 10
Training:


  0%|          | 0/225 [00:00<?, ?it/s]

Train Loss: 0.8168
Train Accuracy: 0.6317
Validating:


  0%|          | 0/25 [00:00<?, ?it/s]

Validate Loss: 0.8134
Validate Accuracy: 0.63
--------------------------------------------------
Epoch 2 of 10
Training:


...

Validate Loss: 0.8034
Validate Accuracy: 0.6425
--------------------------------------------------
Epoch 10 of 10
Training:


  0%|          | 0/225 [00:00<?, ?it/s]

Train Loss: 0.789
Train Accuracy: 0.6497
Validating:


  0%|          | 0/25 [00:00<?, ?it/s]

Validate Loss: 0.8022
Validate Accuracy: 0.6425
--------------------------------------------------
